In [1]:
import csv
from openai import OpenAI
import os
import re

# OPENAI_API_KEYを含むとPushできないため実行時は有効にしてください
# os.environ["OPENAI_API_KEY"] = "*****"

# openAIの機能をclientに代入
client = OpenAI()

# 対象者のMBTIタイプ
user_mbti = "INTJ"

In [2]:
# CSVファイルを読み込み
with open('../output.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    people = list(reader)

In [4]:
def find_best_matches(people, user_mbti, top_n=3):
    
    persons = ""
    for person in people:
        persons = persons + \
            str(f"【名前】 {person['Slack表示名']}, \
                【自己紹介】 {person['自己紹介']}, \
                【得意な業界】 {person['業界']}, \
                【関心のある領域】 {person['関心のある領域']}, \
                【協働で大切にしたいこと】 {person['Tech0のPJTで8期の仲間と協働する際に大切にしたいと思うこと']}, \
                【プロジェクト推進で得意なこと/苦手なこと】 {person['PJTをする上で自分が得意なこと・苦手なこと']}, \
                【参加動機や達成したいこと】 {person['Tech0の参加動機と１年後に到達したい・達成したいこと']},") + "\n"
    
    prompt = f"MBTIタイプが{user_mbti}の人物と合う人物を以下から3名選び、それぞれのMBTIタイプを推測してください。\
            また、対象者とペアでプロレスをする際の必殺技の名前も提案してください。\
            対象者は、次の形式に従って、出力してください。項目には余計な記号や装飾（**など）をつけず、名前はそのまま利用し漢字変換は行わないでください。\
            また、指定した形式に正確に従って出力してください。\
            形式の指定はじまり\n\
                【名前】[名前をそのまま出力]\n\
                【選定理由】[選定理由をそのまま出力]\n \
                【推定MBTI】[推定MBTIをそのまま出力]\n \
                【必殺技】[必殺技の名前をそのまま出力]\n \
            形式の指定終わり\n \
            ここから対象者の情報" + persons

    response =  client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt },
        ],
    )
    
    # コンテンツを抽出
    content = response.choices[0].message.content.strip()  
    print(content)
    
    # 対象者と選定理由を抽出するための正規表現パターン
    pattern = r"【名前】\s*(.*?)\s*【選定理由】\s*(.*?)\s*【推定MBTI】\s*(.*?)\s*【必殺技】\s*(.*?)\s*(?=【名前】|$)"
    matches = re.findall(pattern, content, re.DOTALL)

    # 結果を配列に格納
    result = [{"名前": match[0], "選定理由": match[1], "推定MBTI": match[2], "必殺技": match[3]} for match in matches]

    return result

In [5]:
# 最適な3人を選択
best_matches = find_best_matches(people, user_mbti)

# 結果を表示
for match in best_matches:
     print(match)

【名前】Tnaka Yasuhiro  
【選定理由】Tnaka Yasuhiroはプロジェクト推進や新規事業開発に熱心であり、コミットメントのバランスを取る能力に優れています。この特性から、例えばENTJタイプの人が相性が良いと考えられます。  
【推定MBTI】ENTJ  
【必殺技】フルスロットル・ストラテジックプラン  

【名前】Sugiyama Shoichi  
【選定理由】Sugiyama Shoichiは問題整理や決断力に優れ、また他者とのコミュニケーションを重視しています。これがINTJタイプとの相性を高めており、INFPタイプの人と一緒に働くと良好な関係を築けると考えられます。  
【推定MBTI】INFP  
【必殺技】デシジョン・マスター・コンボ  

【名前】Okabe Yoshihide  
【選定理由】Okabe Yoshihideは人間関係の構築やアイデアの発散が得意であり、コミュニケーションを重視しているため、INTJの計画性と相反する側面を持ったESFJタイプとの協力に適しています。  
【推定MBTI】ESFJ  
【必殺技】コミュニケーション・ハーモニー・アタック
{'名前': 'Tnaka Yasuhiro', '選定理由': 'Tnaka Yasuhiroはプロジェクト推進や新規事業開発に熱心であり、コミットメントのバランスを取る能力に優れています。この特性から、例えばENTJタイプの人が相性が良いと考えられます。', '推定MBTI': 'ENTJ', '必殺技': 'フルスロットル・ストラテジックプラン'}
{'名前': 'Sugiyama Shoichi', '選定理由': 'Sugiyama Shoichiは問題整理や決断力に優れ、また他者とのコミュニケーションを重視しています。これがINTJタイプとの相性を高めており、INFPタイプの人と一緒に働くと良好な関係を築けると考えられます。', '推定MBTI': 'INFP', '必殺技': 'デシジョン・マスター・コンボ'}
{'名前': 'Okabe Yoshihide', '選定理由': 'Okabe Yoshihideは人間関係の構築やアイデアの発散が得意であり、コミュニケーションを重視しているため、INTJの計画性と相反する側面を持ったESFJタイプとの協